In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
# os.environ['tf_cpp_min_log_level'] = '3'

/home/user/.conda/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user/.conda/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user/.conda/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/user/.conda/envs/tensorflow/lib/python3.6/s

In [ ]:
num_epoch = 30
batch_size = 16

In [ ]:
# Make label
path = '/home/user/gift/5class/Machine'  # 改路徑
X = []
y = []
# 已經經過後處理，類似你們的資料夾以迴圈方式存入X_list
# label用0-N(看幾類)代替
for i in os.listdir(path):
    path_im = os.path.join(path, i)
    for j in os.listdir(path_im):
        X.append(os.path.join(path_im, j))
        y.append(int(i))

In [ ]:
# tf.data.Dataset
# 合併
path_img = tf.data.Dataset.from_tensor_slices(X)
label = tf.data.Dataset.from_tensor_slices(y)

In [ ]:
# resize img
def loadImage(img_path):
    image = tf.image.decode_jpeg(tf.io.read_file(img_path), channels=3)
    image /= 255
    image = tf.image.resize(image, [512, 512])
    return image

In [ ]:
# random label
img = path_img.map(lambda path: loadImage(path))
dataset = tf.data.Dataset.zip((img, label))
dataset = dataset.shuffle(len(X))  # 打亂
train_dataset = dataset.take(int(len(X)*.8))  # 80%train
test_dataset = dataset.skip(int(len(X)*.8))  # 20%test
train_dataset_batch = train_dataset.batch(batch_size)
test_dataset_batch = test_dataset.batch(batch_size)
print('data_OK')

data_OK


In [ ]:
#VGG16 in imagenet
model = tf.keras.applications.VGG19(weights='imagenet', include_top=False)
x = model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# x = tf.keras.layers.Dense(1024, activation='relu')(x)
# x = tf.keras.layers.Dense(512, activation='relu')(x)
predictions = tf.keras.layers.Dense(2, activation='softmax')(x)
model = tf.keras.models.Model(inputs=model.input, outputs=predictions)
optimizer = tf.keras.optimizers.Adam(lr=0.00001)

In [ ]:
# train
train_loss_results = []
train_accuracy_results = []

for epoch in range(num_epoch):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    for x, y in train_dataset_batch:
        with tf.GradientTape() as tape:
            predict = model(x, training=True)
            loss = tf.reduce_mean(
                tf.keras.losses.sparse_categorical_crossentropy(y, predict))
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        epoch_loss_avg.update_state(loss)
        epoch_accuracy.update_state(y, model(x, training=True))
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(
        epoch, epoch_loss_avg.result(), epoch_accuracy.result()))
    # save
    #model.save('/home/user/gift/Model/Machine/'+str(epoch+1)+'e_Machine512_VGG19.h5')  # 改路徑

    # test
    test_accuracy = tf.keras.metrics.Accuracy()
    for (x, y) in test_dataset_batch:
        logits = model(x, training=False)
        prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
        test_accuracy(prediction, y)
    print("Test set accuracy: {:.3%}".format(test_accuracy.result()))
    # confusion matrix
    test_img, test_labels = next(iter(test_dataset.batch(len(X)-int(len(X)*.8))))
    predictions = model.predict(test_img)
    confusion_matrix = pd.crosstab(test_labels, tf.argmax(
        predictions, axis=1), rownames=['true'], colnames=['pred'])
    # print(confusion_matrix)

    # f1_score
    Recall_Score = (test_labels, tf.argmax(predictions, axis=1), average='weighted')
    Precision_Score = precision_score(test_labels, tf.argmax(predictions, axis=1), average='weighted')
    Macro_F1 = f1_score(test_labels, tf.argmax(predictions, axis=1), average='weighted')
    print('Recall_Score=', Recall_Score)
    print('Precision_Score=', Precision_Score)
    print('Macro_F1=', Macro_F1)
    print('------------------------------------------------')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 000: Loss: 0.577, Accuracy: 72.170%
Epoch 001: Loss: 0.466, Accuracy: 81.113%
Epoch 002: Loss: 0.416, Accuracy: 83.552%
Epoch 003: Loss: 0.381, Accuracy: 85.741%
Epoch 004: Loss: 0.352, Accuracy: 87.117%
Epoch 005: Loss: 0.323, Accuracy: 87.930%
Epoch 006: Loss: 0.292, Accuracy: 89.181%
Epoch 007: Loss: 0.261, Accuracy: 90.619%
Epoch 008: Loss: 0.230, Accuracy: 91.995%
Epoch 009: Loss: 0.214, Accuracy: 92.933%
Epoch 010: Loss: 0.235, Accuracy: 91.620%
Epoch 011: Loss: 0.195, Accuracy: 94.184%
Epoch 012: Loss: 0.168, Accuracy: 95.059%
Epoch 013: Loss: 0.123, Accuracy: 97.186%
Epoch 014: Loss: 0.096, Accuracy: 98.186%
Epoch 015: Loss: 0.091, Accuracy: 98.186%
Epoch 016: Loss: 0.105, Accuracy: 98.124%
Epoch 017: Loss: 0.090, Accuracy: 98.061%
Epoch 018: Loss: 0.129, Accuracy: 96.811%
Epoch 019: Loss: 0.158, Accuracy: 95.935%
Epoch 020: Loss: 0.134, Accuracy: 96.123%
Epoch 021: Loss: 0.096, 